In [0]:
%tensorflow_version 1.x

# MLP(Multi Layer Perceptron)
### 이해하기  
  * 기존의 perceptron으로는 **XOR Gate**를 해결할 수 없었음
  * XOR 게이트는 AND, OR, NOT gate의 조합으로 만들 수 있음 + perceptron으로는 AND와 OR gate를 만들 수 있음<br>= 여러 개의 perceptron으로 XOR 생성 가능!
<br><br>

### 구조  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![image](https://user-images.githubusercontent.com/38516906/75954862-fe069a00-5ef7-11ea-8420-49f81f4115c0.png)
  * [참고] 용어 설명
    * DNN(Deep Neural Network): Hidden Layer가 3층 이상일 시
      * 딥러닝(Deep Learning)은 Hidden Layer가 3개 이상인 인공 신경망을 의미함
  * [참고] XOR 게이트 세부 설명  
    ![image](https://user-images.githubusercontent.com/38516906/75955237-c51af500-5ef8-11ea-8208-47128bf7759d.png)
  * [참고] layer의 개수에 따라 결정할 수 있는 영역  
   ![image](https://user-images.githubusercontent.com/38516906/75955337-00b5bf00-5ef9-11ea-9959-c3bab8412890.png)

In [8]:
import tensorflow as tf
import numpy as np

# data 생성
x_data = np.array([[0,0], [0,1], [1,0], [1,1]], dtype = np.float32)
y_data = np.array([[0],   [1],   [1],   [0]], dtype = np.float32)



# placeholder 생성. 간단하게 값들을 담는 상자라고 생각하면 됨
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)



# weight와 bias 생성
W1 = tf.Variable(tf.random_normal([2, 2]), name = "weight1") # 행2, 열2인 랜덤 값 생성 / 입력 2개, 출력 2개
b1 = tf.Variable(tf.random_normal([2]), name = "bias1") # w1의 출력이 2개이므로 2로 맞춰줌
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1]), name = "weight2") # 앞에서 출력이 2였기 때문에 2로 맞춰줌
b2 = tf.Variable(tf.random_normal([1]), name = "bias2") # 최종결과는 1개의 출력값
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)



# cost function / minimize cost
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)



# predict / accuracy
predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32) # cast: 조건따라 0,1 반환
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32)) # equal: x,y를 비교하여 boolean값 반환



# start
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for step in range(10001):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if step%1000 == 0:
      print(step, sess.run(cost, feed_dict = {X:x_data, Y:y_data}), sess.run([W1, W2]))
  h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict = {X: x_data, Y: y_data})
  print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.7431668 [array([[ 0.03203906,  0.13442741],
       [ 0.8116213 , -0.7063473 ]], dtype=float32), array([[-0.17556576],
       [ 0.18559927]], dtype=float32)]
1000 0.6929332 [array([[-0.06907163,  0.24811243],
       [ 0.78561455, -0.727968  ]], dtype=float32), array([[-0.37243015],
       [ 0.20722745]], dtype=float32)]
2000 0.6916109 [array([[-0.13653436,  0.488732  ],
       [ 0.7752193 , -0.8092532 ]], dtype=float32), array([[-0.32476747],
       [ 0.42201453]], dtype=float32)]
3000 0.6805143 [array([[-0.1650547 ,  1.1335862 ],
       [ 0.76379526, -1.2344755 ]], dtype=float32), array([[-0.2735066],
       [ 1.0193864]], dtype=float32)]
4000 0.6023091 [array([[-0.11118392,  2.568695  ],
       [ 0.70205426, -2.8343139 ]], dtype=float32), array([[-0.00531138],
       [ 2.5519109 ]], dtype=float32)]
5000 0.5014194 [array([[-0.7277731,  3.9033897],
       [ 1.0780874, -4.3570824]], dtype=float32), array([[1.064168],
       [4.057296]], dtype=float32)]
6000 0.17787896 [array([[-3.406

### 성능 향상
원하는 결과인, Accuracy가 1인 모델을 만들었다.  
하지만 정확도를 더 올리고 싶다면?
1. 첫번째 layer의 결과값을 2보다 큰 수로 늘린다(Wide Neural Network)
   * w1의 [2,2] -> [2,10]
   * b1의 [2] -> [10]
   * w2의 [2,1] -> [10,1]
   * b2의 [1] -> [1]  
   
   그러면 hypothesis의 작은 값은 더 작게, 큰 값은 더 크게 되는 현상을 볼 수 있다.

2. layer를 깊게 쌓는다(Deep Neural Network)
   * 원하는대로 layer 생성: w3 & b3 / w4 & b4 , ...